In [18]:

import pandas as pd
from predict_protein import download_cptac, select_features, train_model
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor, VotingRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, ElasticNetCV

In [3]:
cptac_list = download_cptac.download_cptac(n_tumors=2)

cptac warning: transcriptomics data was not found for the following samples, so transcriptomics data columns were filled with NaN for these samples: C3L-00563.N, C3L-00605.N, C3L-00769.N, C3L-00770.N, C3L-00771.N, C3L-00930.N, C3L-00947.N, C3L-00963.N, C3L-01246.N, C3L-01249.N, C3L-01252.N, C3L-01256.N, C3L-01257.N, C3L-01744.N, C3N-00200.N, C3N-00729.N, C3N-01211.N, NX1.N, NX10.N, NX11.N, NX12.N, NX13.N, NX14.N, NX15.N, NX16.N, NX17.N, NX18.N, NX2.N, NX3.N, NX4.N, NX5.N, NX6.N, NX7.N, NX8.N, NX9.N (/Users/edwardlau/PythonProjects/CPTAC_Protein/predict_protein/download_cptac.py, line 38)
cptac warning: proteomics data was not found for the following samples, so proteomics data columns were filled with NaN for these samples: 01OV002, 01OV008, 01OV010, 01OV013, 01OV019, 01OV045, 01OV046, 01OV049, 02OV035, 02OV040, 02OV042, 02OV045, 04OV041, 11OV009, 13OV004, 17OV019, 17OV034, 22OV001, 26OV010 (/Users/edwardlau/PythonProjects/CPTAC_Protein/predict_protein/download_cptac.py, line 42)
cptac

In [4]:
tm = train_model.LearnCPTAC(pd.concat(cptac_list))

In [5]:
tm.train_method = 'linreg'
tm.included_features = 'single'

In [10]:
tm.shared_proteins[3281]

'GK2'

In [11]:
# This worked in string earlier
res = tm.learn_one_protein('GK2')
res

ValueError: Shape of passed values is (244, 0), indices imply (244, 1)

In [13]:
protein_to_do = 'GK2'
y_df = tm.df[[protein_to_do + '_proteomics']]
y_df = y_df.dropna(subset=[protein_to_do + '_proteomics'])
len(y_df)

103

In [16]:
proteins_to_include = [protein_to_do]
x_df = tm.df[[tx + '_transcriptomics' for tx in proteins_to_include]]
x_df

Name,GK2_transcriptomics
Patient_ID,
OV01OV002,NaN
OV01OV007,NaN
OV01OV008,NaN
OV01OV010,NaN
OV01OV013,NaN
...,...
BRX21BR001,NaN
BRX21BR002,NaN
BRX21BR010,NaN


In [30]:
x_df = x_df.dropna(axis=1, how='all')

In [32]:
x_df.shape[1]

0

In [19]:
x_impute = SimpleImputer(missing_values=np.nan, strategy='median').fit_transform(x_df)
x_impute_df = pd.DataFrame(x_impute, columns=x_df.columns, index=x_df.index)

ValueError: Shape of passed values is (244, 0), indices imply (244, 1)